# To collect the images run the Heatmap_Image_Collection_.ipynb File

In [ ]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 121.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 28.5 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import json
pd.set_option('display.max_rows', None)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Loading in Missing Neighborhood Data

In [ ]:
df_missing_graffiti = pd.read_csv('/content/drive/Shareddrives/UChicago Data for the Common Good/Heatmap/Sample Heatmap/Graffiti_Predictions_Majority_of_Neighborhoods_Missing.csv')
df_missing_graffiti['Prediction'] = df_missing_graffiti['Prediction'].astype(int)
df_missing_graffiti = df_missing_graffiti.groupby(['Neighborhood'])['Prediction'].sum().reset_index()

df_missing_bad_road = pd.read_csv('/content/drive/Shareddrives/UChicago Data for the Common Good/Heatmap/Sample Heatmap/Missing_Neighborhoods.csv')
df_missing_bad_road = df_missing_bad_road.drop(columns = ['Unnamed: 0'])


In [ ]:

missing_dfs = [df_missing_graffiti, df_missing_bad_road]
new_missing_dfs = []
for df_missing, disorder in zip(missing_dfs, ['Graffiti', 'Bad Road']):
  df_missing['Neighborhood'] = df_missing['Neighborhood'].str.capitalize()

  df_missing['Neighborhood'] = df_missing['Neighborhood'].replace('East_village', 'East Village')
  df_missing['Neighborhood'] = df_missing['Neighborhood'].replace('Garfield_park', 'Garfield Park')
  df_missing['Neighborhood'] = df_missing['Neighborhood'].replace('Gold_coast', 'Gold Coast')
  df_missing['Neighborhood'] = df_missing['Neighborhood'].replace('Grant_park', 'Grant Park')
  df_missing['Neighborhood'] = df_missing['Neighborhood'].replace('Jackson_park', 'Jackson Park')
  df_missing['Neighborhood'] = df_missing['Neighborhood'].replace('Little_italy_uic', 'Little Italy, UIC')
  df_missing['Neighborhood'] = df_missing['Neighborhood'].replace('Little_village', 'Little Village')
  df_missing['Neighborhood'] = df_missing['Neighborhood'].replace('Magnificent_mile', 'Magnificent Mile')
  df_missing['Neighborhood'] = df_missing['Neighborhood'].replace('Museum_campus', 'Museum Campus')
  df_missing['Neighborhood'] = df_missing['Neighborhood'].replace('Old_town', 'Old Town')
  df_missing['Neighborhood'] = df_missing['Neighborhood'].replace('Printers_row', 'Printers Row')
  df_missing['Neighborhood'] = df_missing['Neighborhood'].replace('River_north', 'River North')
  df_missing['Neighborhood'] = df_missing['Neighborhood'].replace('Rush_&_division', 'Rush & Division')
  df_missing['Neighborhood'] = df_missing['Neighborhood'].replace('Sauganash,forest_glen', 'Sauganash,Forest Glen')
  df_missing['Neighborhood'] = df_missing['Neighborhood'].replace('Sheffield_&_depaul', 'Sheffield & DePaul')
  df_missing['Neighborhood'] = df_missing['Neighborhood'].replace('Ukrainian_village', 'Ukrainian Village')
  df_missing['Neighborhood'] = df_missing['Neighborhood'].replace('United_center', 'United Center')
  df_missing['Neighborhood'] = df_missing['Neighborhood'].replace('West_loop', 'West Loop')
  df_missing['Neighborhood'] = df_missing['Neighborhood'].replace('Wicker_park', 'Wicker Park')

  df_missing['Neighborhood'] = df_missing['Neighborhood'].str.lower()

  df_missing = df_missing[df_missing['Neighborhood'] != "O'hare"]

  df_missing['Criteria'] = disorder
  df_missing = df_missing.rename(columns = {'Prediction' : 'Count'})

  cols = list(df_missing.columns)

  df_missing = df_missing[cols[:1] + cols[2:] + [cols[1]]]


  df_missing['Neighborhood'] = df_missing['Neighborhood'].str.capitalize()
  new_missing_dfs.append(df_missing)
missing_dfs_all = pd.concat([new_missing_dfs[0], new_missing_dfs[1]], ignore_index=True)
missing_dfs_all['Neighborhood'] = missing_dfs_all['Neighborhood'].str.replace("_", " ").str.title()


#Loading in original Non Missing Road Data

In [ ]:
df_graffiti = pd.read_csv('/content/drive/Shareddrives/UChicago Data for the Common Good/Heatmap/Sample Heatmap/Graffiti_Predictions_Majority_of_Neighborhoods.csv')
df_graffiti['Prediction'] = df_graffiti['Prediction'].astype(int)
df_graffiti = df_graffiti.groupby(['Neighborhood'])['Prediction'].sum().reset_index()
df_graffiti["Neighborhood"] = df_graffiti["Neighborhood"].str.replace("_INTERIOR", "", regex=False)



df_bad_road = pd.read_csv('/content/drive/Shareddrives/UChicago Data for the Common Good/Heatmap/Sample Heatmap/predictions_250.csv')
df_bad_road = df_bad_road.groupby(['Neighborhood'])['Prediction'].sum().reset_index()

#df_bad_road = df_bad_road.drop(columns = ['Unnamed: 0'])

In [ ]:
df_graffiti.head()

,Neighborhood,Prediction
0,ALBANY_PARK,3
1,ARCHER_HEIGHTS,4
2,ARMOUR_SQUARE,4
3,ASHBURN,2
4,AUBURN_GRESHAM,4


In [ ]:
df_bad_road.head()

,Neighborhood,Prediction
0,ALBANY_PARK,14
1,ARCHER_HEIGHTS,12
2,ARMOUR_SQUARE,17
3,ASHBURN,35
4,AUBURN_GRESHAM,16


## Combining the Bad Road and Graffiti data

In [ ]:
# Load the mock data

df_not_missing = [df_bad_road,df_graffiti ]
df_not_missing_new = []
for df, disorder in zip(df_not_missing, ['Bad Road', 'Graffiti']):

  df['Criteria'] = disorder
  df = df.rename(columns = {'Prediction' : 'Count'})
  df['Neighborhood'] = df['Neighborhood'].str.replace("_", " ").str.title()
  #df_grouped['Neighborhood'] = df_grouped['Neighborhood'].replace('East Garfield Part', 'Garfield Park')
  df['Neighborhood'] = df['Neighborhood'].replace('Greater Grand Crossing', 'Grand Crossing')
  df['Neighborhood'] = df['Neighborhood'].replace('Ohare', "O'Hare")
  df['Neighborhood'] = df['Neighborhood'].str.lower()

  df_not_missing_new.append(df)

not_missing_dfs_all = pd.concat([df_not_missing_new[0], df_not_missing_new[1]], ignore_index=True)
not_missing_dfs_all['Neighborhood'] = not_missing_dfs_all['Neighborhood'].str.capitalize()
# df_grouped = pd.concat([df_grouped, df_missing], ignore_index=True)

complete_df = pd.concat([not_missing_dfs_all, missing_dfs_all], ignore_index=True)


# Load the GeoJSON file
geojson_path = '/content/drive/Shareddrives/UChicago Data for the Common Good/Heatmap/Sample Heatmap/Neighborhoods_2012b_20241029.geojson'

with open(geojson_path) as f:
    geojson_data = json.load(f)

# Configuring the GeoJson File

In [ ]:
import json

# Load the GeoJSON file
with open(geojson_path, "r") as f:
    geojson_data = json.load(f)

# Extract neighborhood names (adjust the key based on your GeoJSON structure)
neighborhoods = [feature["properties"]["pri_neigh"] for feature in geojson_data["features"]]

# Display the extracted neighborhood names
print(neighborhoods)


['Grand Boulevard', 'Printers Row', 'United Center', 'Sheffield & DePaul', 'Humboldt Park', 'Garfield Park', 'North Lawndale', 'Little Village', 'Armour Square', 'Avalon Park', 'Burnside', 'Hermosa', 'Avondale', 'Logan Square', 'Calumet Heights', 'East Side', 'West Pullman', 'Garfield Ridge', 'New City', 'Englewood', 'Grand Crossing', 'Ashburn', 'Mount Greenwood', 'Morgan Park', "O'Hare", 'Jackson Park', 'Loop', 'Pullman', 'Riverdale', 'Hegewisch', 'Greektown', 'Douglas', 'Museum Campus', 'Edgewater', 'Lake View', 'Lincoln Park', 'Magnificent Mile', 'Lincoln Square', 'Oakland', 'Grant Park', 'West Loop', 'Fuller Park', 'Andersonville', 'Woodlawn', 'Portage Park', 'Rush & Division', 'Little Italy, UIC', 'Kenwood', 'Rogers Park', 'Jefferson Park', 'Sauganash,Forest Glen', 'North Park', 'Albany Park', 'Irving Park', 'Dunning', 'West Ridge', 'Uptown', 'Norwood Park', 'Streeterville', 'South Shore', 'Chatham', 'South Chicago', 'Roseland', 'North Center', 'South Deering', 'Washington Park', 

## Getting Neighborhood Area

In [ ]:
neighborhood_areas = {}
for i in range(0,len(geojson_data['features'])):
  neighborhood = geojson_data["features"][i]['properties']['pri_neigh']
  area = round(float(geojson_data["features"][i]['properties']['shape_area']), 0)
  neighborhood_areas[neighborhood] = area

neighborhood_areas_df = pd.DataFrame(neighborhood_areas.items(), columns=['community', 'Shape_Area'])


In [ ]:
neighborhood_areas_df

,community,Shape_Area
0,Grand Boulevard,48492503.0
1,Printers Row,2162138.0
2,United Center,32520513.0
3,Sheffield & DePaul,10482592.0
4,Humboldt Park,125010426.0
5,Garfield Park,89976070.0
6,North Lawndale,89487422.0
7,Little Village,127998298.0
8,Armour Square,17141469.0
9,Avalon Park,34852738.0


In [ ]:
complete_df['Neighborhood'] = complete_df['Neighborhood'].str.title()
complete_df['Neighborhood'] = complete_df['Neighborhood'].str.replace(r'\s*Interior$', '', regex=True)

#complete_df = complete_df.merge(neighborhood_areas_df, left_on='Neighborhood', right_on='community', how='left')
#complete_df = complete_df.drop(columns=['community'])
complete_df['Shape_Area'] = complete_df['Shape_Area'].round(0)

complete_df['Weighted_count'] = (complete_df['Count'] / complete_df['Shape_Area']) * 1e8

In [ ]:
complete_df.head()

,Neighborhood,Count,Criteria,Shape_Area,Weighted_count
0,Albany Park,14,Bad Road,53542231.0,26.147584
1,Archer Heights,12,Bad Road,55922506.0,21.458266
2,Armour Square,17,Bad Road,17141469.0,99.174697
3,Ashburn,35,Bad Road,135460337.0,25.837821
4,Auburn Gresham,16,Bad Road,105065354.0,15.228617


## GeoJson Neighborhood Nomenclature

In [ ]:
#Heathmap Taxonomy
import io
temp = pd.read_csv(io.StringIO('''
Neighborhood
Greektown
Avondale
West Lawn
Beverly
Chatham
Magnificent Mile
Fuller Park
West Town
Lincoln Square
Armour Square
East Side
Loop
Grand Crossing
West Ridge
Washington Heights
Rush & Division
"Little Italy, UIC"
Woodlawn
Norwood Park
Chinatown
Auburn Gresham
Sheffield & DePaul
Little Village
Hegewisch
"Sauganash,Forest Glen"
Montclare
Wicker Park
Garfield Park
Mount Greenwood
Albany Park
Uptown
West Elsdon
Portage Park
Oakland
Streeterville
United Center
West Loop
Roseland
Ukrainian Village
North Center
Lincoln Park
Lower West Side
Brighton Park
Hyde Park
Logan Square
North Park
Grand Boulevard
Calumet Heights
Jefferson Park
Morgan Park
Museum Campus
Wrigleyville
Printers Row
Archer Heights
Jackson Park
North Lawndale
Rogers Park
East Village
Andersonville
Gold Coast
Millenium Park
Ashburn
Humboldt Park
Mckinley Park
Douglas
Near South Side
Hermosa
Grant Park
Old Town
Bridgeport
Pullman
Edison Park
Edgewater
O'Hare
River North
Englewood
Burnside
Galewood
Austin
Bucktown
Irving Park
Dunning
Washington Park
South Shore
Kenwood
South Chicago
Clearing
Lake View
South Deering
Chicago Lawn
Belmont Cragin
Boystown
New City
Garfield Ridge
Avalon Park
West Pullman
Gage Park
Riverdale
'''), header=None)


# Creating Heatmap

In [ ]:
import plotly.io as pio

pio.templates.default = "plotly_white"  # Use default styling instead

mapbox_access_token = "open-street-map"  # Get from mapbox.com (Free account)
px.set_mapbox_access_token(mapbox_access_token)


In [ ]:
# Initialize the Dash app
app = dash.Dash(__name__)

unique_criteria = complete_df["Criteria"].unique()  # Use complete_df


# App Layout
app.layout = html.Div([
    html.H1("Chicago Neighborhood Disorder Heatmap"),

    html.Div([
        dcc.Dropdown(
            id="criteria-dropdown",
            options=[{"label": crit, "value": crit} for crit in unique_criteria],
            value=unique_criteria[0] if len(unique_criteria) > 0 else None,  # Default to first available option
            clearable=False,
            style={"width": "50%"}
        )
    ], style={"padding": "20px"}),

    dcc.Graph(id="heatmap"),
])

In [ ]:
@app.callback(
    Output("heatmap", "figure"),
    Input("criteria-dropdown", "value")
)
def update_heatmap(selected_criteria):
    # Filter data
    filtered_df = complete_df[complete_df["Criteria"] == selected_criteria]

    # Create choropleth map
    fig = px.choropleth_mapbox(
        filtered_df,
        geojson=geojson_data,
        locations="Neighborhood",
        featureidkey="properties.pri_neigh",  # Ensure this matches the GeoJSON key
        color="Count",  # Use weighted count for better representation
        color_continuous_scale="Reds",
        mapbox_style="carto-positron",
        zoom=10,
        center={"lat": 41.8781, "lon": -87.6298},  # Center on Chicago
        opacity=0.7,
        labels={"Weighted_count": "Severity"}
    )

    fig.update_layout(margin={"r":0, "t":0, "l":0, "b":0})
    return fig

# Run app
if __name__ == "__main__":
    app.run_server(debug=True)

<IPython.core.display.Javascript object>

## Converting Heatmap to Streamlit App

In [ ]:
#!pip install streamlit
!pip install streamlit pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 122.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 106.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.4 MB/s eta 0:00:00


In [ ]:
complete_df.to_csv('complete_df.csv')


# Define your GeoJSON file path
geojson_path = "saved_geojson.json"

# # Load GeoJSON data
# with open(geojson_path, "r") as f:
#     geojson_data = json.load(f)

# Save the loaded GeoJSON to a new file
with open("backup_geojson.json", "w") as f:
    json.dump(geojson_data, f, indent=4)

print("GeoJSON data has been saved as 'backup_geojson.json'")



GeoJSON data has been saved as 'backup_geojson.json'


In [ ]:
%%writefile app.py

import streamlit as st
import pandas as pd
import plotly.express as px
import json

# Load data (replace with your actual data loading method)
# complete_df = pd.read_csv("your_data.csv")  # Example if data is in a CSV file
complete_df = pd.read_csv("complete_df.csv")
with open("backup_geojson.json", "r") as f:
    geojson_data = json.load(f)

unique_criteria = complete_df["Criteria"].unique()

# Streamlit App Title
st.title("Chicago Neighborhood Disorder Heatmap")

# Dropdown for criteria selection
selected_criteria = st.selectbox("Select Disorder Criteria:", unique_criteria)

# Filter data based on selected criteria
filtered_df = complete_df[complete_df["Criteria"] == selected_criteria]

# Create choropleth map
fig = px.choropleth_mapbox(
    filtered_df,
    geojson=geojson_data,
    locations="Neighborhood",
    featureidkey="properties.pri_neigh",  # Ensure this matches the GeoJSON key
    color="Count",
    color_continuous_scale="Reds",
    mapbox_style="carto-positron",
    zoom=10,
    center={"lat": 41.8781, "lon": -87.6298},  # Center on Chicago
    opacity=0.7,
    labels={"Count": "Disorder Instances"}
)

fig.update_layout(margin={"r":0, "t":0, "l":0, "b":0})

# Display the heatmap in Streamlit
st.plotly_chart(fig)


Overwriting app.py


Create a ngrok account and pull in your authentication token

In [ ]:
!ngrok config add-authtoken YOUR_AUTH_TOKEN


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pkill -f ngrok


In [ ]:
import os
from pyngrok import ngrok

# Kill any existing Streamlit processes
os.system("pkill streamlit")

# Start Streamlit in the background
os.system("streamlit run app.py &")

# Create an ngrok tunnel to Streamlit
public_url = ngrok.connect(addr="8501", proto="http")
print(f"Streamlit App is live at: {public_url}")

Streamlit App is live at: NgrokTunnel: "https://1b4f-35-186-151-69.ngrok-free.app" -> "http://localhost:8501"


***